# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 8 Object References, Mutability, and Recycling

객체는 변수가 할당되기 전에 생성된다. 

참소변수의 경우 변수가 객체에 할당되었다는 표현이 맞다. 



In [1]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))
        
x = Gizmo()
y = Gizmo() * 10

Gizmo id: 140061904230440
Gizmo id: 140061904230496


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [3]:
print(dir())

['Gizmo', 'In', 'Out', '_', '_2', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'quit', 'x']


곱셈을 시도하기전에 두번쨰 Gizmo 객체가 생성.

그러나 곱셈을 시도하고 예외가 발생했음으로 y 변수는 생성되지 않음.


In [6]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis['balance'] = 950

alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

alex == charles # 객체가 동일한 값을 갖고 있음으로 == 연산자에 의해 동일하다고 판단
alex is not charles # 같은 정체성을 갖는건 아니다. they are distinct objects. different identities.

True

True

In [7]:
id(alex), id(charles)

(140062043185872, 140061903605264)

### The Relative Immutability of Tuples

the immutability of tuples really refers to the physical contents of the tuple data structure (i.e., the references it holds), and does not extend to the referenced objects.


In [14]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
id(t1)
id(t2)

140061144344904

140061161282800

In [15]:
t1[-1].append(99)
t1
t1==t2

(1, 2, [30, 40, 99])

False

해시가능하다 : 주기동안 해시값이 변하지 않는다. 일부튜플은 해시 불가능하다.

### Copies Are Shallow by Default

using the constructor or [:] produces a shallow copy

In [17]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = l1[:]
l2 == l1
l2 is l1

True

False

In [18]:
l2[1] is l1[1]

True

### Deep and Shallow Copies of Arbitrary Objects

In [20]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [21]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(140061142229568, 140061142230072, 140061142230688)

* call by sharing :  each formal parameter of the function gets a copy of each reference in the arguments. In other words, the parameters inside the function become **aliases** of the actual arguments

*  you should avoid mutable objects as default values for parameters.

### Defensive Programming with Mutable Parameters

In [16]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    def __init__(self, passengers=[]): 
        self.passengers = passengers
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [17]:
bus2 = HauntedBus()
bus2.pick("A")
bus2.passengers
bus3 = HauntedBus() # 기본값이 들어있음 
bus3.passengers

['A']

['A']

In [18]:
HauntedBus.__init__.__defaults__

(['A'],)

In [19]:
class TwilightBus:
    """A bus model that makes passengers vanish"""
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers # it will be alias of basketball_team 
#             self.passengers = list(passengers) 
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name) 

In [22]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team) 
bus.drop('Pat')
basketball_team # 버스에서 내린 Pat 이 팀에서 사라졌다.

['Sue', 'Tina', 'Maya', 'Diana']

### del and Garbage Collection

del 는 이름을 제거하는 것이지 객체가 제거되지 않는다. 

refcount 객체 참조 개수가 0 이면 GC 

In [23]:
def abc(a, b, c=1):
    print(a, b, c)

In [24]:
abc(1,2,3)

1 2 3


### Weak References

Weak references to an object do not increase its reference count. 

A weak reference is a callable that returns the referenced object or None if the referent is no more

[weakref — Weak references](https://docs.python.org/3/library/weakref.html)

In [26]:
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
wref, wref()

(<weakref at 0x7f220b7dce08; to 'set' at 0x7f220b1f2c88>, {0, 1})

In [27]:
a_set = {2, 3, 4} 
wref()    

{0, 1}

In [30]:
wref() is None

False

In [32]:
wref() is None

False

In [33]:
wref()

{0, 1}

class WeakValueDictionary implements a mutable mapping where the values are weak references to objects. 

When a referred object is garbage collected elsewhere in the program, the corresponding key is automatically removed from WeakValueDictionary. 

This is commonly used for caching.

* interning : The sharing of string literals is an optimization technique 

The fact that variables hold references has many practical consequences in Python programming:
Simple assignment does not create copies.

* Augmented assignment with += or *= creates new objects if the lefthand variable is bound to an immutable object, but may modify a mutable object in place.

* Assigning a new value to an existing variable does not change the object previously bound to it. This is called a rebinding: the variable is now bound to a different object.
If that variable was the last reference to the previous object, that object will be garbage collected.

* Function parameters are passed as aliases, which means the function may change any mutable object received as an argument. There is no way to prevent this, except making local copies or using immutable objects (e.g., passing a tuple instead of a
list).

* Using mutable objects as default values for function parameters is dangerous because if the parameters are changed in place, then the default is changed, affecting every future call that relies on the default.